# Configuración CUDA

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda

In [10]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


In [11]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-534c92b1-d0c4-521f-c011-5c5bb20afca4)


In [0]:
!pip install pycuda

# Specs

In [0]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G  7.6G  333G   3% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G  8.0K  6.4G   1% /var/colab
/dev/sda1       365G   11G  355G   3% /opt/bin
shm             6.0G  4.0K  6.0G   1% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms rtm rdseed adx smap xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: Genuin

# Imports

In [0]:
import numpy as np
import gzip
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt

# PyCUDA imports
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
from pycuda import cumath, gpuarray

# Forward

In [0]:
convolution_mod = SourceModule("""
    __global__ void convolution(float *image,
                                int n_c,
                                int in_dim,
                                float* filt, 
                                int n_f,
                                int n_c_f,
                                int f,
                                int bias,
                                int stride) 
    {

      int out_dim = ((in_dim - f)/stride)+1);
      const int bytes = n_f * out_dim * out_dim;
      
      float* out;
      for(int i=0; i < bytes; i++) {
        out[i] = 0;
      }
      
      for(int i=0; i<n_f; i++) {
        int curr_y = 0;
        int out_y = 0;
        
        while(curr_y + f <= in_dim) {
          int curr_x = 0;
          int out_x = 0;
          
          while(curr_x + f <= in_dim) {
            out[i*out_y*out_x] = 1;
          }
        }
      }
 
    };
    """)

In [0]:
def convolution(image, filt, bias, s=1):
  '''Applies convolutional operation to the image using given filters.
    
  Args:
    image: The input image
    filt: The filters of the convolutional operation.
    bias: A list with the (shared) biases of each filter.
    s: Stride of the convolutional layer
  Returns:
    The output of passing the filters through the convolution (3D image)
  '''
  
  # Get input and output shapes
  (n_f, n_c_f, f, _) = filt.shape
  n_c, in_dim, _ = image.shape 
  out_dim = int((in_dim - f)/s)+1

  # Initialize output as zero
  out = np.zeros((n_f, out_dim, out_dim))

  # For every filter
  for curr_f in range(n_f):
    # Track y index of current (convolutional) and output (next) layer.
    curr_y = 0
    out_y = 0
    
    # Move filter through columns of the image
    while curr_y + f <= in_dim:
      # Track x index of current (convolutional) and output (next) layer.
      curr_x = 0
      out_x = 0
      
      # Move filter through rows of the image
      while curr_x + f <= in_dim:
        # Multiply (element-wise) filters, add result and bias
        out[curr_f, out_y, out_x] = np.sum(filt[curr_f] * image[:,curr_y:curr_y+f, curr_x:curr_x+f]) + bias[curr_f]

        curr_x += s
        out_x += 1
      curr_y += s
      out_y += 1

  return out

def maxpool(image, f=2, s=1):
  '''Downsamples using max-pooling
    
  Args:
    image: The input image
    f: The size of the max-pooling window
    s: Stride of the window
  Returns:
    The output of max-pooling layer.
  '''
  
  # Get input shape
  n_c, h_prev, w_prev = image.shape
  
  # Calculate output shape
  h = int((h_prev - f)/s)+1
  w = int((w_prev - f)/s)+1
  downsampled = np.zeros((n_c, h, w))
  
  # For every channel of the image
  for i in range(n_c):
    # Track y index of current (convolutional) and output (next) layer.
    curr_y = 0
    out_y = 0
    
    # Move filter through columns of the image
    while curr_y + f <= h_prev:
      # Track x index of current (convolutional) and output (next) layer.
      curr_x = 0
      out_x = 0
      
      # Move filter through rows of the image
      while curr_x + f <= w_prev:
        # Find highest value of the original image in the window position
        downsampled[i, out_y, out_x] = np.max(image[i, curr_y:curr_y+f, curr_x:curr_x+f])

        curr_x += s
        out_x += 1
      curr_y += s
      out_y += 1
  return downsampled

# Utils

In [0]:
def extract_data(filename, num_images, image_width=28):
  '''Extracts images using bytestream and reshape to [num_images, height, weight]
  '''
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
      bytestream.read(16)
      buf = bytestream.read(image_width * image_width * num_images)
      data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
      data = data.reshape(num_images, image_width*image_width)
      return data

def extract_labels(filename, num_images):
  '''Extracts label into a vector of [m, 1] where m is the number of images.
  '''
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
      bytestream.read(8)
      buf = bytestream.read(1 * num_images)
      labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  return labels

def initializeFilter(size, scale = 1.0):
  """Initialized a weight from a Gaussian normal distribution.
  """
  stddev = scale/np.sqrt(np.prod(size))
  return np.random.normal(loc = 0, scale = stddev, size = size)

def initializeWeight(size):
  """Iitializes a weight from a standard normal distribution.
  """
  return np.random.standard_normal(size=size) * 0.01
  
def nanargmax(arr):
  """Applies nanargmax for the max-pooling backpropagation.
  
  Example: nanargmax(np.array([0, 0, 0.01, 0, 0])) -> (2,)
  """
  idx = np.nanargmax(arr)
  idxs = np.unravel_index(idx, arr.shape)
  return idxs

def softmax(X):
  """Applies the softmax activation.
  """
  out = np.exp(X)
  return out/np.sum(out)

def categoricalCrossEntropy(probs, label):
  """Calculates the loss with categorical cross entropy.
  """
  return -np.sum(label * np.log(probs))


# Backwards

In [0]:
def convolutionBackward(dconv_prev, conv_in, filt, s):
  '''Backpropagate through the convolutional layer.

  For convolution backpropagation, we need to consider the filter weights, the
  bias and the convolutional operation. 
  
  Each filter shares the bias, making the math simple. For each filter, we only
  need to add all the gradients of the previous layer for the current filter.
  
  To update the filter values, we multiply the value of the gradient at a
  specific point and multiply it with the value of the next layer.
  
  To backpropagate through the convolutional operator (pass the error to the
  previous layer), we need to calculate the gradient for each value. To do this,
  we just multiply the value of the gradient times the value of the filter,
  since it's the weight.
    
  Args:
    d_conv_prev: Gradient respective to this convolutional layer.
    conv_in: The layer to which the convolutional layer is connected to.
    filt: Size of the filter.
    s: Stride of the convolutional layer
  Returns:
    The gradient of the weights, bias, and convolution operation.
  '''
  
  # Get input and output shapes
  (n_f, n_c, f, _) = filt.shape
  (_, orig_dim, _) = conv_in.shape
  
  # Initialize gradients as 0
  dout = np.zeros(conv_in.shape) 
  dfilt = np.zeros(filt.shape)
  dbias = np.zeros((n_f, 1))
  
  # For every filter
  for curr_f in range(n_f):
    # Track y index of current (convolutional) and output (previous) layer.
    curr_y = 0
    out_y = 0

    # Move filter through rows of image
    while curr_y + f <= orig_dim:
      # Track x index of current (maxpooling) and output (previous) layer.
      curr_x = 0
      out_x = 0
      
      # Move filter through columns of the image
      while curr_x + f <= orig_dim:
        # Calculate loss gradient of filter
        dfilt[curr_f] += dconv_prev[curr_f, out_y, out_x] * conv_in[:, curr_y:curr_y+f, curr_x:curr_x+f]
        
        # Calculate loss gradient of convolution operation
        dout[:, curr_y:curr_y+f, curr_x:curr_x+f] += dconv_prev[curr_f, out_y, out_x] * filt[curr_f] 
  
        curr_x += s
        out_x += 1

      curr_y += s
      out_y += 1

    # Update shared bias of the filter
    dbias[curr_f] = np.sum(dconv_prev[curr_f])

  return dout, dfilt, dbias

def maxpoolBackward(dpool, orig, f, s):
  '''Backpropagate through the max-pooling layer.
  
  For max-pooling backpropagation, the gradient of the pooling layer is only
  backpropagated through the highest element of the input at the current window.
  There is no gradient with respect to non-maximum values since they don't
  affect the error. All other values get a gradient of zero. Based on:
  https://datascience.stackexchange.com/questions/11699/backprop-through-max-pooling-layers
    
  Args:
    d_pool: Gradient respective to the pooling layer.
    orig: The layer to which the max-pooling layer is connected.
    f: Size of the pooling window.
    s: Stride of the pooling layer
  Returns:
    The gradient of the pooling layer.
  '''
  (n_c, orig_dim, _) = orig.shape

  dout = np.zeros(orig.shape)

  # Iterate through channels
  for curr_c in range(n_c):
    # Track y index of current (maxpooling) and output (previous) layer.
    curr_y = 0
    out_y = 0

    # Move window through rows of image
    while curr_y + f <= orig_dim:
      # Track x index of current (maxpooling) and output (previous) layer.
      curr_x = 0
      out_x = 0

      # Move window through columns of the image
      while curr_x + f <= orig_dim:
        # Obtain index of the largest value (max pooling)
        (a, b) = nanargmax(orig[curr_c, curr_y:curr_y+f, curr_x:curr_x+f])
        dout[curr_c, curr_y+a, curr_x+b] = dpool[curr_c, out_y, out_x]

        curr_x += s
        out_x += 1
      curr_y += s
      out_y += 1

  return dout

# Network

In [0]:
def network(image, label, params, conv_s, pool_f, pool_s):
  '''Runs an iteration of the full network
    
  Args:
    image: An image from the training set
    label: A one-hot encoded label.
    params: A list containing the filters, weights and biases.
    conv_s: The stride of the convolutional network.
    pool_f: The size of the maxpooling layer.
    pool_s: The stride of the maxpooling layer.
  Returns:
    The cost after the training is complete
  '''
  [f1, f2, w1, w2, b1, b2, b3, b4] = params 

  # Feed forward
  
  # First convolutional layer + ReLU activation
  conv1 = convolution(image, f1, b1)
  conv1[conv1<=0] = 0

  # Second convolutional layer + ReLU activation
  conv2 = convolution(conv1, f2, b2)
  conv2[conv2<=0] = 0

  # Maxpooling
  pooled = maxpool(conv2, pool_f, pool_s)

  # Flattening
  (nf2, dim2, _) = pooled.shape
  fc = pooled.reshape((nf2 * dim2 * dim2, 1))
  
  # First dense layer + ReLU activation
  w1 = gpuarray.to_gpu(w1)
  fc = gpuarray.to_gpu(fc)
  z = gpuarray.dot(w1, fc)
  z = z.get() + b3
  z[z<=0] = 0

  # Output layer (dense) + softmax activation to predict probabilities
  w2 = gpuarray.to_gpu(w2)
  z = gpuarray.to_gpu(z)
  out = gpuarray.dot(w2, z)
  out = out.get() + b4
  probs = softmax(out)

  # Calculate loss with categorical crossentropy
  loss = categoricalCrossEntropy(probs, label)

  # Calculate error
  dout = probs - label 
  
  # Last layer -> first network layer
  # Loss gradient respective to the final dense layer weights and biases
  z = z.get()
  dw2 = dout.dot(z.T)
  db4 = np.sum(dout, axis = 1).reshape(b4.shape)

  # First network layer -> flatten layer
  # Loss gradient respective to the first dense layer weights and biases.
  # Additionally backpropagate through ReLU.
  w2 = w2.get()
  fc = fc.get()
  dz = w2.T.dot(dout)
  dz[z<=0] = 0
  dw1 = dz.dot(fc.T)
  db3 = np.sum(dz, axis = 1).reshape(b3.shape)

  # Flatten layer -> pooling layer
  w1 = w1.get()
  dfc = w1.T.dot(dz)
  # Reshape for correct dimensions of pooling layer
  dpool = dfc.reshape(pooled.shape)

  # Pooling layer -> second convolutional
  # Backpropagate through max-pooling layer and ReLU
  dconv2 = maxpoolBackward(dpool, conv2, pool_f, pool_s)
  dconv2[conv2<=0] = 0

  # Second convolutional -> first convolutional
  # Backpropagate through convolution and ReLU
  dconv1, df2, db2 = convolutionBackward(dconv2, conv1, f2, conv_s)
  dconv1[conv1<=0] = 0

  # First convolutional -> input image
  dimage, df1, db1 = convolutionBackward(dconv1, image, f1, conv_s)
  
  # Get gradients
  grads = [df1, df2, dw1, dw2, db1, db2, db3, db4] 

  # Return gradients and loss
  return grads, loss

# Optimization

In [0]:
def adam(batch, num_classes, lr, dim, n_c, beta1, beta2, params, cost):
  '''Runs the Adam optimizer of the convolutional neural network
    
  Args:
    num_classes: Number of classes for the problem. For this problem, 10 digits.
    lr: Learning rate, also knowns as alpha for Adam.
    n_c: Number of channels of the input images.
    beta1: Exponential decay rate of first momentum.
    beta2: Exponential decay rate of second momentum.
    params: A list containing the filters, weights and biases.
    cost: A list of costs.

  Returns:
    The updated parameters and the updated list of costs.
  '''
  [f1, f2, w1, w2, b1, b2, b3, b4] = params

  # Get input and reshape 
  X = batch[:,0:-1]
  X = X.reshape(len(batch), n_c, dim, dim)
  Y = batch[:,-1]

  # Cost of this iteration
  cost_ = 0

  batch_size = len(batch)

  # Initialize gradients 
  df1 = np.zeros(f1.shape)
  df2 = np.zeros(f2.shape)
  dw1 = np.zeros(w1.shape)
  dw2 = np.zeros(w2.shape)
  db1 = np.zeros(b1.shape)
  db2 = np.zeros(b2.shape)
  db3 = np.zeros(b3.shape)
  db4 = np.zeros(b4.shape)

  # Initialize momentum
  v1 = np.zeros(f1.shape)
  v2 = np.zeros(f2.shape)
  v3 = np.zeros(w1.shape)
  v4 = np.zeros(w2.shape)
  bv1 = np.zeros(b1.shape)
  bv2 = np.zeros(b2.shape)
  bv3 = np.zeros(b3.shape)
  bv4 = np.zeros(b4.shape)

  # Initialize RMS param
  s1 = np.zeros(f1.shape)
  s2 = np.zeros(f2.shape)
  s3 = np.zeros(w1.shape)
  s4 = np.zeros(w2.shape)
  bs1 = np.zeros(b1.shape)
  bs2 = np.zeros(b2.shape)
  bs3 = np.zeros(b3.shape)
  bs4 = np.zeros(b4.shape)

  # Iterate through elements
  for i in range(batch_size):
    # Extract the element
    x = X[i]
    
    # One-hot encoding
    y = np.eye(num_classes)[int(Y[i])].reshape(num_classes, 1)

    # Get gradient and loss
    grads, loss = network(x, y, params, 1, 2, 4)
    [df1_, df2_, dw1_, dw2_, db1_, db2_, db3_, db4_] = grads

    # Update gradients (get cumulative)
    df1 += df1_
    db1 += db1_
    
    df2 += df2_
    db2 += db2_
    
    dw1 += dw1_
    db3 += db3_
    
    dw2 += dw2_
    db4 += db4_

    cost_+= loss

  # Update momentum, RMSProp and combine to perform Adam update
  v1 = beta1*v1 + (1-beta1)*df1/batch_size 
  s1 = beta2*s1 + (1-beta2)*(df1/batch_size)**2 
  f1 -= lr * v1/np.sqrt(s1+1e-7) 

  bv1 = beta1*bv1 + (1-beta1)*db1/batch_size
  bs1 = beta2*bs1 + (1-beta2)*(db1/batch_size)**2
  b1 -= lr * bv1/(np.sqrt(bs1)+1e-7)

  v2 = beta1*v2 + (1-beta1)*df2/batch_size
  s2 = beta2*s2 + (1-beta2)*(df2/batch_size)**2
  f2 -= lr * v2/(np.sqrt(s2)+1e-7)

  bv2 = beta1*bv2 + (1-beta1) * db2/batch_size
  bs2 = beta2*bs2 + (1-beta2)*(db2/batch_size)**2
  b2 -= lr * bv2/(np.sqrt(bs2)+1e-7)

  v3 = beta1*v3 + (1-beta1) * dw1/batch_size
  s3 = beta2*s3 + (1-beta2)*(dw1/batch_size)**2
  w1 -= lr * v3/(np.sqrt(s3)+1e-7)

  bv3 = beta1*bv3 + (1-beta1) * db3/batch_size
  bs3 = beta2*bs3 + (1-beta2)*(db3/batch_size)**2
  b3 -= lr * bv3/(np.sqrt(bs3)+1e-7)

  v4 = beta1*v4 + (1-beta1) * dw2/batch_size
  s4 = beta2*s4 + (1-beta2)*(dw2/batch_size)**2
  w2 -= lr * v4/(np.sqrt(s4)+1e-7)

  bv4 = beta1*bv4 + (1-beta1)*db4/batch_size
  bs4 = beta2*bs4 + (1-beta2)*(db4/batch_size)**2
  b4 -= lr * bv4/(np.sqrt(bs4)+1e-7)

  # Update cost and store
  cost_ = cost_/batch_size
  cost.append(cost_)

  # Parameters
  params = [f1, f2, w1, w2, b1, b2, b3, b4]

  return params, cost


# Training

In [0]:
def train(num_classes = 10, lr = 0.01, beta1 = 0.95, beta2 = 0.99, img_dim = 28, img_depth = 1, f = 2, num_filt1 = 4, num_filt2 = 4, batch_size = 64, num_epochs = 1, save_path = 'params.pkl'):
  '''Executes the training of the optimization.
  
  This function loads the features and labels, initializes the weights and
  filters, and splits the data in batches to later be used in every iteration
  that calls the Adam optimization. Reports the results in a dump file.
    
  Args:
    num_classes: Number of classes for the problem. For this problem, 10 digits.
    lr: Learning rate, also knowns as alpha for Adam.
    beta1: Exponential decay rate of first momentum.
    beta2: Exponential decay rate of second momentum.
    img_dim: Dimensions of the image, assuming it is a square image.
    img_depth: Number of channels of the image
    f: Size of the filter.
    num_filt1: Number of filters in the first convolutional layer.
    num_filt2: Number of filters in the second convolutional layer.
    batch_size: Number of training samples to be used in every iteration.
    num_epochs: Times to iterate through the whole training set.
    save_path: Path to save the parameters.

  Returns:
    A list of costs.
  '''
  
  # Number of training samples
  m = 50000

  # Get training data
  X = extract_data('train-images-idx3-ubyte.gz', m, img_dim)
  y = extract_labels('train-labels-idx1-ubyte.gz', m).reshape(m, 1)
  
  # Normalize data
  X -= int(np.mean(X))
  X /= int(np.std(X))
  train_data = np.hstack((X, y))

  np.random.shuffle(train_data)

  # Initialize filters. The are the number of dimensions x number of channels x the size of the filter
  f1 = (num_filt1, img_depth, f, f)
  f2 = (num_filt2, num_filt1, f, f)
  
  # Weights of the neural network. 2500 (flatten) -> 128 -> 10
  w1 = (128, 196)
  w2 = (10, 128)
  
  # Initialize filter and weight values
  f1 = initializeFilter(f1)
  f2 = initializeFilter(f2)
  w1 = initializeWeight(w1)
  w2 = initializeWeight(w2)

  # Initialize the bias with a value of 0
  b1 = np.zeros((f1.shape[0],1))
  b2 = np.zeros((f2.shape[0],1))
  b3 = np.zeros((w1.shape[0],1))
  b4 = np.zeros((w2.shape[0],1))

  # Combine params
  params = [f1, f2, w1, w2, b1, b2, b3, b4]
  cost = []

  print("LR:" +str(lr)+ ", Batch Size:" + str(batch_size))

  for epoch in range(num_epochs):
    # Shuffle data and divide in batches
    np.random.shuffle(train_data)
    batches = [train_data[k: k+batch_size] for k in range(0, train_data.shape[0], batch_size)]

    # Feed every batch to the Adam Gradient Descent algorithm
    t = tqdm(batches, position=0)
    for x, batch in enumerate(t):
        params, cost = adam(batch, num_classes, lr, img_dim, img_depth, beta1, beta2, params, cost)
        t.set_description("Cost: %.2f" % (cost[-1]))

  # Save relevant data in the file
  data = [params, cost]
  with open(save_path, 'wb') as file:
      pickle.dump(data, file)

  return cost

# Prediction

In [0]:
def predict(image, f1, f2, w1, w2, b1, b2, b3, b4):
  '''Makes predictions from given parameters.

  Args:
    image: An image of 1x28x28 of a digit.
    f1: A list of filters of the first convolutional layer.
    f2: A list of filters of the second convolutional layer.
    w1: The weights of the first neural layer.
    w2: The weights of the second neural layer.
    b1-b4: The biases of the 4 layers.

  Returns:
    A tuple with the digit predicted and the probability of the image being that digit.
  '''

  # First convolution
  conv1 = convolution(image, f1, b1)
  conv1[conv1<=0] = 0

  # Second convolution
  conv2 = convolution(conv1, f2, b2) 
  conv2[conv2<=0] = 0 

  # Max poooling
  pooled = maxpool(conv2, 2, 4) 

  # Flattening
  (nf2, dim2, _) = pooled.shape
  fc = pooled.reshape((nf2 * dim2 * dim2, 1))

  # First dense layer
  w1 = gpuarray.to_gpu(w1)
  fc = gpuarray.to_gpu(fc)
  z = gpuarray.dot(w1, fc)
  z = z.get() + b3
  z[z<=0] = 0

  # Output layer
  w2 = gpuarray.to_gpu(w2)
  z = gpuarray.to_gpu(z)
  out = gpuarray.dot(w2, z)
  out = out.get() + b4
  probs = softmax(out)

  # Return the maximum value given the probabilities
  return np.argmax(probs), np.max(probs)

# Main

In [0]:
# Train
cost = train()

# Load parameters from file
params, cost = pickle.load(open('params.pkl', 'rb'))
[f1, f2, w1, w2, b1, b2, b3, b4] = params

# Draw plot of loss
plt.plot(cost, 'r')
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.legend('Loss', loc='upper right')
plt.show()

# Get test data
m = 10000
X = extract_data('t10k-images-idx3-ubyte.gz', m, 28)
y = extract_labels('t10k-labels-idx1-ubyte.gz', m).reshape(m,1)

# Normalize data
X -= int(np.mean(X))
X /= int(np.std(X))

test_data = np.hstack((X, y))

# Extract features and be sure that the structure is 1x28x28
X = test_data[:,0:-1]
X = X.reshape(len(test_data), 1, 28, 28)

# Extract label
y = test_data[:,-1]

# Counter of correct predictions
correct = 0
print("Computing accuracy over test set:")

t2 = tqdm(range(len(X)), position=0)
for i in t2:
    x = X[i]
    
    # Get a prediction. Here prob would be useful for other metrics.
    pred, prob = predict(x, f1, f2, w1, w2, b1, b2, b3, b4)

    if pred == y[i]:
        correct+=1

    t2.set_description("Acc:%0.2f%%" % (float(correct/(i+1))*100))

print("Overall Accuracy: %.2f" % (float(correct/len(test_data)*100)))